In [1]:
!python -V

Python 3.12.7


In [2]:
import pandas as pd
import pickle
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_extraction import DictVectorizer
import mlflow

In [3]:
df1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")

In [4]:
df1.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00


In [5]:
print(df1.lpep_dropoff_datetime - df1.lpep_pickup_datetime)
df1['duration'] = df1.lpep_dropoff_datetime - df1.lpep_pickup_datetime
df1.duration = df1.duration.apply(lambda td: td.total_seconds()/60)

print(df1.duration)

0       0 days 00:03:56
1       0 days 00:08:45
2       0 days 00:05:58
3       0 days 00:07:05
4       0 days 00:00:04
              ...      
76513   0 days 00:38:00
76514   0 days 00:38:00
76515   0 days 00:11:00
76516   0 days 00:27:00
76517   0 days 00:10:00
Length: 76518, dtype: timedelta64[us]
0         3.933333
1         8.750000
2         5.966667
3         7.083333
4         0.066667
           ...    
76513    38.000000
76514    38.000000
76515    11.000000
76516    27.000000
76517    10.000000
Name: duration, Length: 76518, dtype: float64


In [6]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("NYC_FarePredictor")

<Experiment: artifact_location='/Users/muhammadwisalabdullah/PycharmProjects/MLOPsZoomcampHWs/HW3/mlruns/1', creation_time=1748937303092, experiment_id='1', last_update_time=1748937303092, lifecycle_stage='active', name='NYC_FarePredictor', tags={}>

In [9]:
def read_dataframe1(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[df.duration >= 1] & (df.duration <= 60)

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


In [ ]:
train_df = read_dataframe1("/Users/muhammadwisalabdullah/Downloads/yellow_tripdata_2023-01.parquet")
valid_df = read_dataframe1("/Users/muhammadwisalabdullah/Downloads/yellow_tripdata_2023-02.parquet")